In [1]:
import numpy as np
import pandas as pd
import json
import re
import os
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
from huggingface_hub import login
from google.colab import userdata

In [4]:
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
login(token=HUGGINGFACE_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import transformers
import torch

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    max_length=1024
)

ompound='Sc'
output_dir='/content/drive/MyDrive/Fine-tuning-llms/test_output/'

messages = [
    {"role": "system", "content": "You are a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations."},
    {"role": "user", "content": "Please, generate an input file for Sc compound."},
]

outputs = pipeline(
    messages,
    max_new_tokens=1024,
)
print(outputs[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations.'}, {'role': 'user', 'content': 'Please, generate an input file for Sc compound.'}, {'role': 'assistant', 'content': "To generate a Quantum Espresso input file for a single point energy calculation of Sc (Scandium), we'll need to define the atomic structure, pseudopotential, and calculation settings. Here's an example input file:\n\n```pseudopotential\n&control\n  calculation ='scf',\n  restart_mode = 'from_scratch',\n  prefix ='sc',\n  pseudo_dir = '/path/to/pseudopotentials',\n  outdir = './sc_out',\n  verbosity = 'high',\n/\n\n&system\n  ibrav = 2,\n  celldm(1) = 6.3,\n  nat = 1,\n  ntyp = 1,\n  ecutwfc = 30,\n  ecutrho = 240,\n  occupations = 'fixed',\n  degauss = 0.01,\n  smearing = 'none',\n  nosym =.true.,\n  nosym_evc =.true.,\n  input_dynamics = 'none',\n/\n\n&electrons\n  conv_thr = 1.0d-8,\n  electron_maxstep

In [8]:
mes=outputs[0]["generated_text"][-1]
print(mes['content'])

To generate a Quantum Espresso input file for a single point energy calculation of Sc (Scandium), we'll need to define the atomic structure, pseudopotential, and calculation settings. Here's an example input file:

```pseudopotential
&control
  calculation ='scf',
  restart_mode = 'from_scratch',
  prefix ='sc',
  pseudo_dir = '/path/to/pseudopotentials',
  outdir = './sc_out',
  verbosity = 'high',
/

&system
  ibrav = 2,
  celldm(1) = 6.3,
  nat = 1,
  ntyp = 1,
  ecutwfc = 30,
  ecutrho = 240,
  occupations = 'fixed',
  degauss = 0.01,
  smearing = 'none',
  nosym =.true.,
  nosym_evc =.true.,
  input_dynamics = 'none',
/

&electrons
  conv_thr = 1.0d-8,
  electron_maxstep = 100,
  mixing_beta = 0.7,
  mixing_ndim = 6,
  mixing_mode = 'plain',
  bremix = 1,
  diagonalization = 'david',
  pseudo_dir = '/path/to/pseudopotentials',
/

ATOMIC_SPECIES
  Sc  44.955910  Sc.pbe.UPF
/

ATOMIC_POSITIONS (angstrom)
  Sc  0.0 0.0 0.0
/

CELL_PARAMETERS (angstrom)
  6.3 0.0 0.0
  0.0 6.3 0.0
  0

In [11]:
with open('/content/drive/MyDrive/Fine-tuning-llms/data_unaries/1.in','r') as f:
    input_file1=f.read()
with open('/content/drive/MyDrive/Fine-tuning-llms/data_oxides/10.in','r') as f:
    input_file2=f.read()

In [17]:
print(input_file2)

&CONTROL
  calculation = 'scf'
  etot_conv_thr =   1.5000000000d-05
  forc_conv_thr =   5.0000000000d-05
  max_seconds =   6.8400000000d+03
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  restart_mode = 'from_scratch'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   4.5000000000d-03
  ecutrho =   6.0000000000d+02
  ecutwfc =   7.5000000000d+01
  ibrav = 0
  nat = 3
  nosym = .false.
  ntyp = 2
  occupations = 'smearing'
  smearing = 'fd'
/
&ELECTRONS
  conv_thr =   3.0000000000d-10
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
ATOMIC_SPECIES
Al     26.981538 Al.pbe-n-kjpaw_psl.1.0.0.UPF
O      15.9994 O.pbe-n-kjpaw_psl.0.1.UPF
ATOMIC_POSITIONS angstrom
Al           4.2145368473       1.4048456158       4.2145368473 
Al           4.2145368473       1.4048456158       1.4048456158 
O            0.0000000000       0.0000000000       0.0000000000 
K_POINTS automatic
33 33 33 0 0 0
CELL_PARAMETERS angstrom
      0.00000000

In [18]:
messages = [
    {"role": "system", "content": "You are a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations."},
    {"role": "user", "content": "Please, generate an input file for Sc compound. \
    The example of the input file is: " +input_file2+ "Analysise the content and generate the similar file for Sc."},
]

In [19]:
outputs = pipeline(
    messages,
    max_new_tokens=2048,
)
mes=outputs[0]["generated_text"][-1]
print(mes['content'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The provided input file is a Quantum Espresso input file for a single point energy calculation of the AlO compound. Here's a breakdown of the content:

1.  **&CONTROL**: This section contains parameters related to the control of the calculation.
    *   `calculation ='scf'`: This specifies that the calculation is a self-consistent field (SCF) calculation.
    *   `etot_conv_thr` and `forc_conv_thr`: These are the convergence thresholds for the total energy and force, respectively.
    *   `max_seconds`: This is the maximum time allowed for the calculation in seconds.
    *   `outdir` and `prefix`: These specify the output directory and prefix for the output files, respectively.
    *   `pseudo_dir`: This is the directory where the pseudopotentials are located.
    *   `restart_mode`: This specifies whether to restart the calculation from a previous run or start from scratch.
    *   `tprnfor` and `tstress`: These are flags that control the output of the forces and stresses, respectivel